# Run workflow
The following script outlines the steps to ingest miniscope metadata and processed data into the DataJoint workflow-miniscope.  

In [1]:
import numpy as np
import os
os.chdir('..')
from workflow_miniscope.pipeline import *

DataJoint verified plugin `datajoint_connection_hub` detected.
Connecting dbadmin@djhub-hosting-ucla-001.clv9vocx3mqz.us-west-2.rds.amazonaws.com:3306


## Insert entries into the following manual DataJoint tables

+ subject.Subject
+ experiment.Equipment
+ experiment.Session
+ scan.Scan
+ imaging.ProcessingTask

As described in the following sections, the entries can be inserted manually, or with the `ingest` method and accompanying `csv` files.

In [12]:
subject.Subject.insert1(dict(subject='subject1', 
                             sex='F', 
                             subject_birth_date='2019-01-01 00:00:01', 
                             subject_description='no description'))

Equipment.insert1(dict(scanner='MINI2P_ImagingScanner'))

Session.insert1(dict(subject='subject1', 
                     session_datetime='2021-01-01 00:00:01'))

Session.Directory.insert1(dict(subject='subject1', 
                               session_datetime='2021-01-01 00:00:01', 
                               session_dir='<imaging_root_data_dir>/subject1/session0'))

scan.Scan.insert1(dict(subject='subject1', 
                       session_datetime='2021-01-01 00:00:01', 
                       scan_id=0, 
                       scanner='MINI2P_ImagingScanner', 
                       scan_notes=''))

### Method for inserting entries

Modify `user_data/subjects.csv` and `user_data/sessions.csv`, and run the following commands

In [2]:
from workflow_miniscope.ingest import ingest_subjects, ingest_sessions

In [3]:
ingest_subjects()


---- Insert 1 entry(s) into subject.Subject ----

---- Successfully completed ingest_subjects ----


In [4]:
ingest_sessions()


---- Insert 1 entry(s) into experiment.Equipment ----

---- Insert 1 entry(s) into session.Session ----

---- Insert 1 entry(s) into scan.Scan ----

---- Successfully completed ingest_sessions ----


In [5]:
populate_settings = {'display_progress': True, 'reserve_jobs': False, 'suppress_errors': False}

scan.ScanInfo.populate(**populate_settings)

ScanInfo: 100%|██████████| 1/1 [00:00<00:00,  1.22it/s]


## Insert new ProcessingTask to trigger ingestion of motion-correction/segmentation results

Motion correction and segmentation are performed for each scan,
once the processing job has been completed, an entry in the `ProcessingTask` needs to be added to trigger the ingestion of the processing results.

Two pieces of information need to be specified:
+ the `paramset_idx` used for the processing job
+ the output directory storing the processing results

In [6]:
scan.Scan()

subject,session_datetime,scan_id,scanner,acq_software,scan_notes free-notes
pingping_miniscope,2021-02-12 16:21:34.121000,0,Miniscope-DAQ-V3,Miniscope-DAQ-V3,


In [7]:
import pathlib
from workflow_miniscope.paths import get_imaging_root_data_dir

root_dir = pathlib.Path(get_imaging_root_data_dir())

In [8]:
for scan_key in (scan.Scan & scan.ScanInfo - imaging.ProcessingTask).fetch('KEY'):
    scan_file = root_dir / (scan.ScanInfo.ScanFile & scan_key).fetch('file_path')[0]
    recording_dir = scan_file.parent

    # MiniscopeAnalysis
    miniscope_analysis_dir = recording_dir / 'miniscope_analysis'
    if miniscope_analysis_dir.exists():
        imaging.ProcessingTask.insert1({**scan_key,
                                        'paramset_idx': 0,
                                        'processing_output_dir': miniscope_analysis_dir.as_posix()})

In [9]:
imaging.ProcessingTask()

subject,session_datetime,scan_id,paramset_idx,processing_output_dir output directory of the processed scan relative to root data directory,"task_mode 'load': load computed analysis results, 'trigger': trigger computation"
pingping_miniscope,2021-02-12 16:21:34.121000,0,0,/Users/kabilar/Documents/Data/U24/imaging_sample_data/pingping_miniscope/session0/miniscope_analysis,load


In [10]:
imaging.Processing.populate(**populate_settings)

Processing: 100%|██████████| 1/1 [00:00<00:00,  1.96it/s]


## Insert new Curation following the ProcessingTask

In this example, we create/insert one Curation for each ProcessingTask, specifying the same output directory

To this end, we make use of a convenient function `imaging.Curation().create1_from_processing_task()`

In [11]:
for key in (imaging.ProcessingTask - imaging.Curation).fetch('KEY'):
    imaging.Curation().create1_from_processing_task(key)

## Populate the following imported and computed DataJoint tables

+ scan.ScanInfo
+ imaging.Processing
+ imaging.MotionCorrection
+ imaging.Segmentation
+ imaging.MaskClassification
+ imaging.Fluorescence
+ imaging.Activity

As described in the following sections, populating the tables can be done individually or with the `populate` method.

In [ ]:
populate_settings = {'display_progress': True, 'reserve_jobs': False, 'suppress_errors': False}

scan.ScanInfo.populate(**populate_settings)
imaging.Processing.populate(**populate_settings)
imaging.MotionCorrection.populate(**populate_settings)
imaging.Segmentation.populate(**populate_settings)
imaging.MaskClassification.populate(**populate_settings)
imaging.Fluorescence.populate(**populate_settings)
imaging.Activity.populate(**populate_settings)

### Method for populating tables

In [2]:
from workflow_miniscope.populate import populate

In [3]:
populate(display_progress=False)


---- Populate scan.ScanInfo ----

---- Populate imaging.Processing ----

---- Populate imaging.MotionCorrection ----

---- Populate imaging.Segmentation ----

---- Populate imaging.MaskClassification ----

---- Populate imaging.Fluorescence ----

---- Populate imaging.Activity ----
3
4
5
6
*subject       *session_datet *scan_id    *paramset_idx  *curation_id   *extraction_me processing_met
+------------+ +------------+ +---------+ +------------+ +------------+ +------------+ +------------+
pingping_minis 2021-02-12 16: 0           0              1              miniscope_anal miniscope_anal
pingping_minis 2021-02-12 16: 0           0              1              miniscope_anal miniscope_anal
 (Total: 2)

*subject       *session_datet *scan_id    *paramset_idx  *curation_id   *extraction_me
+------------+ +------------+ +---------+ +------------+ +------------+ +------------+
pingping_minis 2021-02-12 16: 0           0              1              miniscope_anal
pingping_minis 2021-02-12 

## Drop schemas
If required to drop all schemas, the following is the dependency order.

In [5]:
from workflow_miniscope.pipeline import *

In [6]:
imaging.schema.drop()
scan.schema.drop()
session.schema.drop()
subject.schema.drop()
lab.schema.drop()